In [1]:
library(corrplot)
library(PerformanceAnalytics)
library(ggplot2)
library(FactoMineR)
library(factoextra)
library(dplyr)
library(lattice)
library(cluster)
library(visreg)
library(car)
library(tidyr)
library(caret)
library(forcats) # fct_recode function
library(repr)    # for figure size
library(randomForest)

corrplot 0.84 loaded
Loading required package: xts
Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric

Registered S3 method overwritten by 'xts':
  method     from
  as.zoo.xts zoo 

Attaching package: ‘PerformanceAnalytics’

The following object is masked from ‘package:graphics’:

    legend

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Welcome! Want to learn more? See two factoextra-related books at https://goo.gl/ve3WBa

Attaching package: ‘dplyr’

The following objects are masked from ‘package:xts’:

    first, last

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: carData

Attaching package: ‘car’

The following object is masked from ‘package:dplyr’:

    

In [2]:
is.Date <- function(x) inherits(x, 'Date')

In [3]:
# library(summarytools)

# Traitement des features

## Variables pas vraiment numériques -> conversion en date

- YearBuilt
- YearRemodAdd
- GarageYrBlt
- MoSold
- YrSold

In [4]:
trainfull$YearBuilt <- as.Date(
    ISOdate(trainfull[, 'YearBuilt'], 01, 01), format = "%Y")
trainfull$YearRemodAdd <- as.Date(
    ISOdate(trainfull[, 'YearRemodAdd'], 01, 01), format = "%Y")
trainfull$GarageYrBlt <- as.Date(
    ISOdate(trainfull[, 'GarageYrBlt'], 01, 01), format = "%Y")
trainfull$YrSold <- as.Date(
    ISOdate(trainfull[, 'YrSold'], trainfull[, 'MoSold'], 01), format = "%Y")

trainfull = select(trainfull, -MoSold)

ERROR: Error in vapply(list(year, month, day, hour, min, sec), length, 1, USE.NAMES = FALSE): object 'trainfull' not found


## Variable non quantitative -> conversion en qualitative

In [ ]:
function no-qual_conversion(dataframe){
    dataframe$MSSubClass <- factor(dataframe$MSSubClass)
}

## Variables ordinales transformables :

|    Feature       |       Levels      | Score Possibilty  |
|:-----------------|:-----------------:|:-----------------:|
|LotShape                    |irregular -> regular        |1 to 4|
|LandContour                 |depression -> flat           |1 to 4|
|Utilities                   |electricity only -> all      |1 to 4|
|LandSlope                   |sever -> gentle              |1 to 3|
|ExterQual                   |poor -> excellent            |1 to 5|
|ExterCond                   |poor -> excellent            |1 to 5|
|BsmtQual                    |poor -> excellent            |1 to 5|
|BsmtCond                    |poor -> excellent            |1 to 5|
|BsmtExposure                |no -> good                   |1 to 4|
|BsmtFinType1                |unfinished -> good           |1 to 6|
|BsmtFinType2                |unfinished -> good           |1 to 6|    
|HeatingQC                   |poor -> excellent            |1 to 5|
|CentralAir                  |no / yes                     |0 / 1 |
|Electrical                  |poor -> standard             |1 to 5|
|KitchenQual                 |poor -> excellent            |1 to 5|
|Functional                  |salvage -> typical           |1 to 8|
|FireplaceQu                 |poor -> excellent            |1 to 5|
|GarageFinish                |unfinished -> finshed        |1 to 3|
|GarageQual                  |poor -> excellent            |1 to 5|
|GarageCond                  |poor -> excellent            |1 to 5|
|PavedDrive                  |dirt -> paved                |1 to 3|
|PoolQC                      |fair -> excellent            |1 to 4|
|Fence                       |none -> good                 |1 to 5|

In [ ]:
trainfull$LotShape <- fct_recode(trainfull$LotShape,
                   '1' = 'IR3', '2' = 'IR2', '3' = 'IR1', '4' = 'Reg')
trainfull$LotShape <- as.numeric(as.character(trainfull$LotShape))

In [ ]:
trainfull$LandContour <- trainfull$LandContour %>%
    fct_recode('1' = 'Low', '2' = 'HLS', '3' = 'Bnk', '4' = 'Lvl')
trainfull$LandContour <- as.numeric(as.character(trainfull$LandContour))

In [ ]:
trainfull$Utilities <- trainfull$Utilities %>%
    fct_recode('1' = 'ELO', '2' = 'NoSeWa', '3' = 'NoSewr', '4' = 'AllPub')
trainfull$Utilities <- as.numeric(as.character(trainfull$Utilities))

In [ ]:
trainfull$LandSlope <- trainfull$LandSlope %>%
    fct_recode('1' = 'Sev', '2' = 'Mod', '3' = 'Gtl')
trainfull$LandSlope <- as.numeric(as.character(trainfull$LandSlope))

In [ ]:
trainfull$ExterQual <- trainfull$ExterQual %>%
    fct_recode('1' = 'Po', '2' = 'Fa', '3' = 'TA', '4' = 'Gd', '5' = 'Ex')
trainfull$ExterQual <- as.numeric(as.character(trainfull$ExterQual))

In [ ]:
trainfull$ExterCond <- trainfull$ExterCond %>%
    fct_recode('1' = 'Po', '2' = 'Fa', '3' = 'TA', '4' = 'Gd', '5' = 'Ex')
trainfull$ExterCond <- as.numeric(as.character(trainfull$ExterCond))

In [ ]:
trainfull$BsmtQual <- trainfull$BsmtQual %>%
    fct_recode('1' = 'Po', '2' = 'Fa', '3' = 'TA', '4' = 'Gd', '5' = 'Ex')
trainfull$BsmtQual <- as.numeric(as.character(trainfull$BsmtQual))

In [ ]:
trainfull$BsmtCond  <- trainfull$BsmtCond  %>%
    fct_recode('1' = 'Po', '2' = 'Fa', '3' = 'TA', '4' = 'Gd', '5' = 'Ex')
trainfull$BsmtCond  <- as.numeric(as.character(trainfull$BsmtCond))

In [ ]:
trainfull$BsmtExposure  <- trainfull$BsmtExposure  %>%
    fct_recode('1' = 'No', '2' = 'Mn', '3' = 'Av', '4' = 'Gd')
trainfull$BsmtExposure  <- as.numeric(as.character(trainfull$BsmtExposure))

In [ ]:
trainfull$BsmtFinType1  <- trainfull$BsmtFinType1  %>%
    fct_recode('1' = 'Unf', '2' = 'LwQ', '3' = 'Rec',
               '4' = 'BLQ', '5' = 'ALQ', '6' = 'GLQ')
trainfull$BsmtFinType1  <- as.numeric(as.character(trainfull$BsmtFinType1))

In [ ]:
trainfull$BsmtFinType2  <- trainfull$BsmtFinType2  %>%
    fct_recode('1' = 'Unf', '2' = 'LwQ', '3' = 'Rec',
               '4' = 'BLQ', '5' = 'ALQ', '6' = 'GLQ')
trainfull$BsmtFinType2  <- as.numeric(as.character(trainfull$BsmtFinType2))

In [ ]:
trainfull$HeatingQC  <- trainfull$HeatingQC  %>%
    fct_recode('1' = 'Po', '2' = 'Fa', '3' = 'TA', '4' = 'Gd', '5' = 'Ex')
trainfull$HeatingQC  <- as.numeric(as.character(trainfull$HeatingQC))

In [ ]:
trainfull$CentralAir  <- trainfull$CentralAir  %>%
    fct_recode('0' = 'N', '1' = 'Y')
trainfull$CentralAir  <- as.numeric(as.character(trainfull$CentralAir))

In [ ]:
trainfull$Electrical  <- trainfull$Electrical  %>%
    fct_recode('1' = 'Mix', '2' = 'FuseP', '3' = 'FuseF', '4' = 'FuseA', '5' = 'SBrkr')
trainfull$Electrical  <- as.numeric(as.character(trainfull$Electrical))

In [ ]:
trainfull$KitchenQual  <- trainfull$KitchenQual  %>%
    fct_recode('1' = 'Po', '2' = 'Fa', '3' = 'TA', '4' = 'Gd', '5' = 'Ex')
trainfull$KitchenQual  <- as.numeric(as.character(trainfull$KitchenQual))

In [ ]:
trainfull$Functional <- trainfull$Functional  %>%
    fct_recode('1' = 'Sal', '2' = 'Sev', '3' = 'Maj2', '4' = 'Maj1',
               '5' = 'Mod', '6' = 'Min2', '7' = 'Min1', '8' = 'Typ')
trainfull$Functional <- as.numeric(as.character(trainfull$Functional))

In [ ]:
#trainfull$FireplaceQu  <- trainfull$FireplaceQu  %>%
#    fct_recode('1' = 'Po', '2' = 'Fa', '3' = 'TA', '4' = 'Gd', '5' = 'Ex')
#trainfull$FireplaceQu  <- as.numeric(as.character(trainfull$FireplaceQu))

In [ ]:
#trainfull$GarageFinish  <- trainfull$GarageFinish  %>%
#    fct_recode('1' = 'Unf', '2' = 'RFn', '3' = 'Fin')
#trainfull$GarageFinish  <- as.numeric(as.character(trainfull$GarageFinish))

In [ ]:
trainfull$GarageQual  <- trainfull$GarageQual  %>%
    fct_recode('1' = 'Po', '2' = 'Fa', '3' = 'TA', '4' = 'Gd', '5' = 'Ex')
trainfull$GarageQual  <- as.numeric(as.character(trainfull$GarageQual))

In [ ]:
trainfull$GarageCond  <- trainfull$GarageCond  %>%
    fct_recode('1' = 'Po', '2' = 'Fa', '3' = 'TA', '4' = 'Gd', '5' = 'Ex')
trainfull$GarageCond  <- as.numeric(as.character(trainfull$GarageCond))

In [ ]:
trainfull$PavedDrive  <- trainfull$PavedDrive  %>%
    fct_recode('1' = 'N', '2' = 'P', '3' = 'Y')
trainfull$PavedDrive  <- as.numeric(as.character(trainfull$PavedDrive))

In [ ]:
#trainfull$PoolQC  <- trainfull$PoolQC  %>%
#    fct_recode('1' = 'Po', '2' = 'Fa', '3' = 'TA', '4' = 'Gd', '5' = 'Ex')
#trainfull$PoolQC  <- as.numeric(as.character(trainfull$PoolQC))

In [ ]:
#trainfull$Fence  <- trainfull$Fence  %>%
#    fct_recode('1' = 'MnWw', '2' = 'GdWo', '3' = 'MnPrv', '4' = 'GdPrv')
#trainfull$Fence  <- as.numeric(as.character(trainfull$Fence))

## Modalités avec peu de données

In [ ]:
function near_zero(dataframe){
    nzv <- nearZeroVar(dataframe, saveMetrics = TRUE)
    nzv[which(nzv$nzv == TRUE | nzv$zeroVar == TRUE),]
}

## Features avec beaucoup de 0 (3rd quartile = 0)

- BsmtFinSF2
- LowQualFinSF
- EnclosedPorch
- X3SsnPorch
- ScreenPorch
- PoolArea
- MiscVal

other nearzero alerts

- LandContour
- LandSlope
- BsmtCond
- BsmtFinType2
- KitchenAbvGr
- Functional
- GarageQual
- GarageCond

In [ ]:
trainfull <- select(trainfull,
                    - BsmtFinSF2,
                    - LowQualFinSF,
                    - EnclosedPorch,
                    - X3SsnPorch,
                    - ScreenPorch,
                    - PoolArea,
                    - MiscVal,
                    - LandContour,
                    - LandSlope,
                    - BsmtCond,
                    - BsmtFinType2,
                    - KitchenAbvGr,
                    - Functional,
                    - GarageQual,
                    - GarageCond
                   )

## Détails 


|    Feature       |       Modalité    |     Possibilty         |
|:-----------------|:-----------------:|:-----------------------|
|Street            |gravel = 6         |-> supprimer feature    |
|Utilities         |Mode = 1459        |-> supprimer feature    |
|LotConfig         |3frontages = 4     |-> supprimer lignes     |
|Condition2        |mode = 1445/1460   |-> supprimer feature    |
|RoofMatl          |Mode = 1434/1460   |-> RoofMatl 0/1         |
|Foundation        |Wood= 3, stone= 6  |-> supprimer lignes     |
|Heating           |Mode = 1428/1460   |-> convertir en 0/1     |
|Electrical        |Mix=1, fuseP=1     |-> supprimer lignes (finalement conservé car converti en numérique)    |
|GarageType        |2Types=6, carport=9|-> supprimer lignes     |
|MiscFeature       |Mode = 1406/1460   |-> convertir en 0/1     |
|SaleCondition     |AdjLand=4          |-> supprimer lignes     |




In [ ]:
function details(dataframe){
    dataframe <- select(dataframe, -Street, -Utilities, -Condition2)
    
    dataframe$Heating  <- dataframe$Heating  %>%
    fct_recode('0' = 'GasA',
               '1' = 'Floor', '1' = 'GasW', '1' = 'Grav',
               '1' = 'OthW', '1' = 'Wall')
    dataframe$Heating  <- as.numeric(as.character(dataframe$Heating))
    
    dataframe$RoofMatl  <- dataframe$RoofMatl  %>%
    fct_recode('0' = 'CompShg',
               '1' = 'ClyTile', '1' = 'Membran', '1' = 'Metal',
               '1' = 'Roll', '1' = 'Tar&Grv', '1' = 'WdShake', '1' = 'WdShngl')
    dataframe$RoofMatl  <- as.numeric(as.character(dataframe$RoofMatl))
    
    levels(dataframe$MiscFeature) <- c(levels(dataframe$MiscFeature), 'None')
    dataframe$MiscFeature[which(is.na(dataframe$MiscFeature))] <- 'None'
    dataframe$MiscFeature <- dataframe$MiscFeature  %>%
    fct_recode('0' = 'None',
               '1' = 'Elev', '1' = 'Gar2', '1' = 'Othr',
               '1' = 'Shed', '1' = 'TenC')
    dataframe$MiscFeature <- as.numeric(as.character(dataframe$MiscFeature))
}

## Données manquantes

|NA's  |Feature      | Possibility |
|-----:|:------------|:------------|
|  257 |LotFrontage  |-> supprimer lignes
| 1350 |Alley        |-> Créer modalité None
|    8 |MasVnrType   |-> supprimer
|    8 |MasVnrArea   |-> supprimer
|   37 |BsmtQual     |-> supprimer
|   36 |BsmtCond     |-> supprimer
|   37 |BsmtExposure |-> supprimer
|   36 |BsmtFinType1 |-> supprimer
|   37 |BsmtFinType2 |-> supprimer
|    1 |Electrical   |-> supprimer
|  673 |FireplaceQu  |-> Créer modalité None
|   78 |GarageType   |-> supprimer feature si corrélée à une autre
|   78 |GarageYrBlt  |-> mettre la valeur de YearBuilt
|   78 |GarageFinish |-> Créer modalité None
| 1430 |PoolQC       |-> Créer modalité None
| 1161 |Fence        |-> Créer modalité None

In [ ]:
levels(trainfull$Alley) <- c(levels(trainfull$Alley), 'None')
trainfull$Alley[which(is.na(trainfull$Alley))] <- 'None'

In [ ]:
levels(trainfull$FireplaceQu) <- c(levels(trainfull$FireplaceQu), 'None')
trainfull$FireplaceQu[which(is.na(trainfull$FireplaceQu))] <- 'None'

In [ ]:
levels(trainfull$GarageType) <- c(levels(trainfull$GarageType), 'None')
trainfull$GarageType[which(is.na(trainfull$GarageType))] <- 'None'

In [ ]:
levels(trainfull$GarageFinish) <- c(levels(trainfull$GarageFinish), 'None')
trainfull$GarageFinish[which(is.na(trainfull$GarageFinish))] <- 'None'

In [ ]:
levels(trainfull$PoolQC) <- c(levels(trainfull$PoolQC), 'None')
trainfull$PoolQC[which(is.na(trainfull$PoolQC))] <- 'None'

In [ ]:
levels(trainfull$Fence) <- c(levels(trainfull$Fence), 'None')
trainfull$Fence[which(is.na(trainfull$Fence))] <- 'None'

In [ ]:
#trainfull <- trainfull[which(!is.na(trainfull$LotFrontage)),]
#trainfull <- trainfull[which(!is.na(trainfull$MasVnrType)),]
#trainfull <- trainfull[which(!is.na(trainfull$MasVnrArea)),]
#trainfull <- trainfull[which(!is.na(trainfull$BsmtQual)),]
#trainfull <- trainfull[which(!is.na(trainfull$BsmtExposure)),]
#trainfull <- trainfull[which(!is.na(trainfull$BsmtFinType1)),]
#trainfull <- trainfull[which(!is.na(trainfull$Electrical)),]

In [ ]:
NA_lines = which(is.na(trainfull$GarageYrBlt))
trainfull$GarageYrBlt[NA_lines] <- trainfull$YearBuilt[NA_lines]

## Remarques :

Features avec queue de distribution -> possibilité passer en log (si pas de 0)

- BsmtFinSF1
- BsmtUNFSF
- GrLivArea
- GarageArea
- LotArea
- LotFrontage
- MasVnrArea
- OpenPorchSF
- SalePrice
- TotalBsmtSF
- WoodDeckSF
- X1stFlrSF
- X2ndFlrSF

"""
trainfull <- trainfull %>%
    #mutate(BsmtFinSF1=log(BsmtFinSF1)) %>%
    #mutate(BsmtUNFSF=log(BsmtUNFSF)) %>%
    mutate_if(GrLivArea=log(GrLivArea)) %>%
    mutate(GarageArea=log(GarageArea)) %>%
    mutate(LotArea=log(LotArea)) %>%
    mutate(LotFrontage=log(LotFrontage)) %>%
    mutate(MasVnrArea=log(MasVnrArea)) %>%
    mutate(OpenPorchSF=log(OpenPorchSF)) %>%
    mutate(SalePrice=log(SalePrice)) %>%
    mutate(TotalBsmtSF=log(TotalBsmtSF)) %>%
    mutate(WoodDeckSF=log(WoodDeckSF)) %>%
    mutate(X1stFlrSF=log(X1stFlrSF)) %>%
    mutate(X2ndFlrSF=log(X2ndFlrSF))
"""

Beaucoup de valeur à 0 qui semblent anormale -> mettre en NA / supprimer lignes / créer nouvelle feature ?
- BsmtFinSF1
- BsmtFinSF2
- MasVnrArea
- OpenPorchSF
- WoodDeckSF
- X2ndFlrSF